# Datasets

A [Dataset](https://nexus-forge.readthedocs.io/en/latest/interaction.html#dataset) is a specialization of a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) that provides users with operations to handle files, record their provenance and describe them with metadata.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

Note: DemoStore doesn't implement file operations yet. Use the BluBrainNexus store instead when creating a config file.

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

In [4]:
from kgforge.specializations.resources import Dataset

In [5]:
import pandas as pd

## Creation with files

In [6]:
! ls -p ../../data | egrep -v /$

associations.tsv
my_data.xwz
my_data_derived.txt
persons.csv
tfidfvectorizer_model_schemaorg_linking


In [7]:
persons = Dataset(forge, name="Interesting Persons")

In [8]:
persons.add_files("../../data/persons.csv")

In [9]:
forge.register(persons)

<action> _register_one
<succeeded> True


In [10]:
forge.as_json(persons)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/3c2a5b45-d6e3-47f2-a417-4215f600cf72',
 'type': 'Dataset',
 'hasPart': {'distribution': {'type': 'DataDownload',
   'atLocation': {'type': 'Location',
    'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault',
     'type': 'DiskStorage',
     '_rev': 1}},
   'contentSize': {'unitCode': 'bytes', 'value': 52},
   'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/a6d5ebef-246b-4483-8d4c-05e610b68d9a',
   'digest': {'algorithm': 'SHA-256',
    'value': '1dacd765946963fda4949753659089c5f532714b418d30788bedddfec47a389f'},
   'encodingFormat': 'text/csv',
   'name': 'persons.csv'}},
 'name': 'Interesting Persons'}

In [11]:
associations = Dataset(forge, name="Associations data")

In [12]:
associations.add_files("../../data/associations.tsv")

In [13]:
associations.add_derivation(persons)

In [14]:
forge.register(associations)

<action> _register_one
<succeeded> True


In [15]:
forge.as_json(associations)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/dc84f1db-228e-43be-9051-91adc0a3a159',
 'type': 'Dataset',
 'derivation': {'type': 'Derivation',
  'entity': {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/3c2a5b45-d6e3-47f2-a417-4215f600cf72?rev=1',
   'type': 'Dataset',
   'name': 'Interesting Persons'}},
 'hasPart': {'distribution': {'type': 'DataDownload',
   'atLocation': {'type': 'Location',
    'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault',
     'type': 'DiskStorage',
     '_rev': 1}},
   'contentSize': {'unitCode': 'bytes', 'value': 477},
   'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/ad0d4ef3-b11f-46cd-b113-17b4836a9d99',
   'digest': {'algorithm': 'SHA-256',
    'value': '789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859'},
   'encodingFormat': 'text/tab-separated-values',
   'name': 'associations.tsv'}},
 'name': 'Associations data'}

In [19]:
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp.

# A cross_bucket argument can be provided to download data from the configured bucket (cross_bucket=False - the default value) 
# or from a bucket different than the configured one (cross_bucket=True). The configured store should support crossing buckets for this to work.
associations.download(path="./downloaded/", source="parts")

In [20]:
! ls -l ./downloaded

total 8
-rw-r--r--  1 mfsy  staff  477 Mar 22 12:36 associations.tsv


In [21]:
# ! rm -R ./downloaded/

## Creation with resources as parts

In [22]:
distribution_1 = forge.attach("../../data/associations.tsv")

In [23]:
distribution_2 = forge.attach("../../data/persons.csv")

In [24]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution_1)

In [25]:
john = Resource(type="Person", name="John Smith", distribution=distribution_2)

In [26]:
persons = [jane, john]

In [27]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [28]:
dataset = Dataset(forge, name="Interesting people")

In [29]:
dataset.add_parts(persons)

In [30]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [31]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/0723b9d6-47e6-456e-a9b8-ca509c2458e6',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/b8365a08-15ae-40ff-a957-d6da5d15e461?rev=1',
   'type': 'Person',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/705398f8-cbec-4b78-88a6-73b320b27850'},
   'name': 'Jane Doe'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/32ef3bf9-7976-4b78-ba30-b1fd51d78d91?rev=1',
   'type': 'Person',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/e36dcd6e-d381-476c-a51b-541f45dff749'},
   'name': 'John Smith'}],
 'name': 'Interesting people'}

In [32]:
dataset.download(path="./downloaded/", source="parts")

In [33]:
! ls -l ./downloaded

total 24
-rw-r--r--  1 mfsy  staff  477 Mar 22 12:36 associations.tsv
-rw-r--r--  1 mfsy  staff  477 Mar 22 12:36 associations.tsv.20220322123653
-rw-r--r--  1 mfsy  staff   52 Mar 22 12:36 persons.csv


In [33]:
# ! rm -R ./downloaded/

### Creation from resources

In [34]:
dataset = Dataset.from_resource(forge, [jane, john], store_metadata=True)
print(*dataset, sep="\n")

{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/b8365a08-15ae-40ff-a957-d6da5d15e461
    type: Person
    distribution:
    {
        type: DataDownload
        atLocation:
        {
            type: Location
            store:
            {
                id: https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault
                type: DiskStorage
                _rev: 1
            }
        }
        contentSize:
        {
            unitCode: bytes
            value: 477
        }
        contentUrl: https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/705398f8-cbec-4b78-88a6-73b320b27850
        digest:
        {
            algorithm: SHA-256
            value: 789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859
        }
        encodingFormat: text/tab-separated-values
        name: associations.tsv
    }
    name: Jane Doe
}
{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/32ef3bf9-7976-4b

## Creation from a dataframe

See notebook `07 DataFrame IO.ipynb` for details on conversions of instances of Resource from a Pandas DataFrame.

### basics

In [35]:
dataframe = pd.read_csv("../../data/persons.csv")

In [36]:
dataframe

,type,name
0,Person,Marie Curie
1,Person,Albert Einstein


In [37]:
persons = forge.from_dataframe(dataframe)

In [38]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [39]:
dataset = Dataset(forge, name="Interesting people")

In [40]:
dataset.add_parts(persons)

In [41]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [42]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/5cfc04fa-35d2-4cd3-bf11-fb1540a10b71',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/474cd3a0-f8b1-474d-b82d-fbd181114ff8?rev=1',
   'type': 'Person',
   'name': 'Marie Curie'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/8d301671-138b-469e-98c4-17058556edeb?rev=1',
   'type': 'Person',
   'name': 'Albert Einstein'}],
 'name': 'Interesting people'}

### advanced

In [43]:
dataframe = pd.read_csv("../../data/associations.tsv", sep="\t")

In [44]:
dataframe

,id,name,type,agent__type,agent__name,agent__gender__id,agent__gender__type,agent__gender__label,distribution
0,(missing),Curie Association,Association,Person,Marie Curie,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,../../data/scientists-database/marie_curie.txt
1,(missing),Einstein Association,Association,Person,Albert Einstein,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,../../data/scientists-database/albert_einstein...


In [45]:
dataframe["distribution"] = dataframe["distribution"].map(lambda x: forge.attach(x))

In [46]:
associations = forge.from_dataframe(dataframe, na="(missing)", nesting="__")

In [47]:
print(*associations, sep="\n")

{
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000383
            type: LabeledOntologyEntity
            label: female
        }
        name: Marie Curie
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/marie_curie.txt', None])
    name: Curie Association
}
{
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000384
            type: LabeledOntologyEntity
            label: male
        }
        name: Albert Einstein
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/albert_einstein.txt', None])
    name: Einstein Association
}


In [48]:
forge.register(associations)

<count> 2
<action> _register_many
<succeeded> True


In [49]:
dataset = Dataset(forge, name="Interesting associations")

In [50]:
dataset.add_parts(associations)

In [51]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [52]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/521dd053-38d8-4f6d-b47d-c7c502faae86',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/899ca1b8-80d0-4d2e-b5d2-d9586365a00b?rev=1',
   'type': 'Association',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/eab86c1b-2d9c-484c-981b-b4822518fb78'},
   'name': 'Curie Association'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/708ac4d5-05f9-487f-9147-cc14881428e4?rev=1',
   'type': 'Association',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/bd10baa6-737c-4cb6-bd73-06f3f2cb34ec'},
   'name': 'Einstein Association'}],
 'name': 'Interesting associations'}